In [6]:
##  Retrieving GO annotations and import packages ###
import Bio.UniProt.GOA as GOA
from goatools import obo_parser # Import the OBO parser from GOATools
import wget
import os
from ftplib import FTP

# Downloading the obo file

go_obo_url = 'http://purl.obolibrary.org/obo/go/go-basic.obo'
data_folder = os.getcwd() + '/data' # Creating a new directory named data to store obo file.


# Check if the file exists already
if(not os.path.isfile(data_folder+'/go-basic.obo')):
    go_obo = wget.download(go_obo_url, data_folder+'/go-basic.obo')
else:
    go_obo = data_folder+'/go-basic.obo' # Store the go obo file in the variable go_obo
    
# To create dictionary of GO terms.
go = obo_parser.GODag(go_obo)

/Users/timothyfisher/data/go-basic.obo: fmt(1.2) rel(2019-04-17) 47,398 GO Terms


In [7]:
# Download the reduced GAF file containing gene association of only humans. #

'''
Human UniProt-GOA GAF file
ftp://ftp.ebi.ac.uk/pub/databases/GO/goa/HUMAN/goa_human.gaf.gz
'''

human_url = '/pub/databases/GO/goa/HUMAN/goa_human.gaf.gz'
human_fn = human_url.split('/')[-1]

# Check if the file exists already
human_gaf = os.path.join(data_folder, human_fn)
if(not os.path.isfile(human_gaf)):
    # Login to FTP server
    ebi_ftp = FTP('ftp.ebi.ac.uk')
    ebi_ftp.login() # To loing in as random person... Ghose
    
    # Download
    with open(human_gaf,'wb') as human_fp:
        ebi_ftp.retrbinary('RETR {}'.format(human_url), human_fp.write)
        
    # Logout from FTP server
    ebi_ftp.quit()


In [8]:
import gzip
# File is a gunzip file, so we need to open it in this way
with gzip.open(human_gaf, 'rt') as human_gaf_fp:
    human_funcs = {}  # Initialize the dictionary of functions
    
    # Iterate on each function using Bio.UniProt.GOA library.
    for entry in GOA.gafiterator(human_gaf_fp):
        uniprot_id = entry.pop('DB_Object_ID')
        human_funcs[uniprot_id] = entry

In [9]:
### GO enrichment analysis ###
from goatools.go_enrichment import GOEnrichmentStudy

# The reference population from the observed Human GOA file. 
pop = human_funcs.keys()

# The Swiss-Prot gene IDs from mygene_code.py results
study = ['P03905','P00846','P03891','P00395','P00403','P00156']#cluster5_c0c1

# To create a dictionary of genes with their 
# UniProt ID as a key and their set of GO annotations as the values.
assoc = {}
for x in human_funcs:
    if x not in assoc:
        assoc[x] = set()
    assoc[x].add(str(human_funcs[x]['GO_ID']))

## The GO terms that are most significantly enriched. 
# Methods for GOEnrichmentStudy
methods = ["bonferroni", "sidak", "holm", "fdr"]




# Running GOEnrichmentStudy
g = GOEnrichmentStudy(pop, assoc, go,
                         propagate_counts=True,
                         alpha=0.05,
                         methods=methods)
g_res = g.run_study(study)
    

fisher module not installed.  Falling back on scipy.stats.fisher_exact


Propagating term counts to parents ..


1 GO IDs in assc. are not found in the GO-DAG: GO:0008565
100% 19,770 of 19,770 population items found in association
100%      6 of      6 study items found in association
100%      6 of      6 study items found in population(19770)
Calculating 9,143 uncorrected p-values using fisher_scipy_stats
   9,143 GO terms are associated with 19,766 of 19,770 population items
      46 GO terms are associated with      6 of      6 study items
       6 GO terms found significant (< 0.05=alpha) after multitest correction: local bonferroni
       6 GO terms found significant (< 0.05=alpha) after multitest correction: local sidak
       6 GO terms found significant (< 0.05=alpha) after multitest correction: local holm


Generate p-value distribution for FDR based on resampling (this might take a while)
Sample 0 / 500: p-value 0.00030349013657378235


fisher module not installed.  Falling back on scipy.stats.fisher_exact


Sample 10 / 500: p-value 0.00030349013657378235
Sample 20 / 500: p-value 0.00030349013657378235
Sample 30 / 500: p-value 0.00030349013657378235
Sample 40 / 500: p-value 0.00030349013657378235
Sample 50 / 500: p-value 0.00030349013657378235
Sample 60 / 500: p-value 0.0013265401740797564
Sample 70 / 500: p-value 0.000606903514014989
Sample 80 / 500: p-value 0.0033341723136107453
Sample 90 / 500: p-value 0.0009102401479174321
Sample 100 / 500: p-value 0.00030349013657378235
Sample 110 / 500: p-value 0.00030349013657378235
Sample 120 / 500: p-value 0.000277390413074762
Sample 130 / 500: p-value 0.00030349013657378235
Sample 140 / 500: p-value 0.000606903514014989
Sample 150 / 500: p-value 0.000606903514014989
Sample 160 / 500: p-value 0.00030349013657378235
Sample 170 / 500: p-value 0.00030349013657378235
Sample 180 / 500: p-value 0.00030349013657378235
Sample 190 / 500: p-value 0.00045350729635886065
Sample 200 / 500: p-value 0.00030349013657378235
Sample 210 / 500: p-value 0.000303490136

      17 GO terms found significant (< 0.05=alpha) after multitest correction: local fdr


In [10]:
g.print_results(g_res, min_ratio=None, pval=0.05)

GO	NS	enrichment	name	ratio_in_study	ratio_in_pop	p_uncorrected	depth	study_count	p_bonferroni	p_sidak	p_holm	p_fdr	study_items
GO:0055093	BP	e	response to hyperoxia         	1/6	1/19770	0.000303	5	1	1	1	1	0.046	         P00156
GO:0036296	BP	e	response to increased oxygen levels	1/6	1/19770	0.000303	4	1	1	1	1	0.046	         P00156
GO:0072593	BP	e	reactive oxygen species metabolic process	1/6	5/19770	0.00152	3	1	1	1	1	0.862	         P03891
GO:0070482	BP	e	response to oxygen levels     	1/6	10/19770	0.00303	3	1	1	1	1	0.962	         P00156
GO:0009628	BP	e	response to abiotic stimulus  	1/6	74/19770	0.0223	2	1	1	1	1	0.998	         P00156
GO:0015078	MF	e	proton transmembrane transporter activity	3/6	25/19770	3.56e-08	7	3	0.000326	0.000318	0.000326	0	P00395, P00403, P00846
GO:0015077	MF	e	monovalent inorganic cation transmembrane transporter activity	3/6	49/19770	2.85e-07	6	3	0.0026	0.00254	0.0026	0	P00395, P00403, P00846
GO:0022890	MF	e	inorganic cation transmembrane transporter activity	3/